In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
!pip install -U transformers
!pip install datasets
!pip install -U accelerate

# Tokenisation

In [ ]:
from transformers import AutoTokenizer
import torch
from datasets import Dataset, DatasetDict

In [ ]:
BASE_MODEL = "xlm-roberta-base"
LEARNING_RATE = 2e-5
MAX_LEN = 512
BATCH_SIZE = 32
EPOCHS = 4

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

In [ ]:
raw_train_df = pd.read_csv("/content/train_df.csv")
raw_val_df = pd.read_csv("/content/validation_df.csv")
raw_test_df = pd.read_csv("/content/test_df.csv")

In [ ]:
ende_test_df = raw_test_df.iloc[:5456]
zhen_test_df = raw_test_df.iloc[5457:]

In [ ]:
train_mean = raw_train_df["mqm_score"].mean()
train_sd = raw_train_df["mqm_score"].std()

train_mean, train_sd

(1.6212953965061356, 2.085992053944654)

In [ ]:
raw_train_df['s_mqm'] = (raw_train_df['mqm_score'] - train_mean) / train_sd
raw_val_df['s_mqm'] = (raw_val_df['mqm_score'] - train_mean) / train_sd
raw_test_df['s_mqm'] = (raw_test_df['mqm_score'] - train_mean) / train_sd

ende_test_df['s_mqm'] = (ende_test_df['mqm_score'] - train_mean) / train_sd
zhen_test_df['s_mqm'] = (zhen_test_df['mqm_score'] - train_mean) / train_sd

<ipython-input-15-8ee57776921d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ende_test_df['s_mqm'] = (ende_test_df['mqm_score'] - train_mean) / train_sd
<ipython-input-15-8ee57776921d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zhen_test_df['s_mqm'] = (zhen_test_df['mqm_score'] - train_mean) / train_sd


In [ ]:
raw_train_ds = Dataset.from_pandas(raw_train_df)
raw_val_ds = Dataset.from_pandas(raw_val_df)
raw_test_ds = Dataset.from_pandas(raw_test_df)
raw_ende_test_ds = Dataset.from_pandas(ende_test_df)
raw_zhen_test_ds = Dataset.from_pandas(zhen_test_df)

In [ ]:
ds = {"train": raw_train_ds, "validation": raw_val_ds, "test": raw_test_ds}

In [ ]:
def preprocess_function(df):
    score = df['s_mqm']
    df = tokenizer(df["source"], df["target"], truncation=True, padding="max_length", max_length=MAX_LEN)
    df["label"] = float(score)
    return df

In [ ]:
for split in ds:
    ds[split] = ds[split].map(preprocess_function, remove_columns=['Unnamed: 0', 'source', 'seg_id', 'target', 'doc_id', 'system', 'doc',
       'rater', 'severity', 'category_Accuracy/Addition',
       'category_Accuracy/Mistranslation', 'category_Accuracy/Omission',
       'category_Accuracy/Source language fragment',
       'category_Fluency/Character encoding', 'category_Fluency/Grammar',
       'category_Fluency/Inconsistency', 'category_Fluency/Punctuation',
       'category_Fluency/Register', 'category_Fluency/Spelling',
       'category_Locale convention/Currency format',
       'category_Locale convention/Date format',
       'category_Locale convention/Time format', 'category_No-error',
       'category_Other', 'category_Source error', 'category_Style/Awkward',
       'category_Terminology/Inappropriate for context',
       'category_Terminology/Inconsistent',
       'category_Locale convention/Name format', 'category_Non-translation!',
       'category_Locale convention/Address format',
       'category_Locale convention/Telephone format', 'mqm_score', 's_mqm'])

Map:   0%|          | 0/44249 [00:00<?, ? examples/s]

Map:   0%|          | 0/14751 [00:00<?, ? examples/s]

Map:   0%|          | 0/14751 [00:00<?, ? examples/s]

In [ ]:
single_lang = {"ende": raw_ende_test_ds, "zhen": raw_zhen_test_ds}
for split in single_lang:
  single_lang[split] = single_lang[split].map(preprocess_function, remove_columns=['Unnamed: 0', 'source', 'seg_id', 'target', 'doc_id', 'system', 'doc',
       'rater', 'severity', 'category_Accuracy/Addition',
       'category_Accuracy/Mistranslation', 'category_Accuracy/Omission',
       'category_Accuracy/Source language fragment',
       'category_Fluency/Character encoding', 'category_Fluency/Grammar',
       'category_Fluency/Inconsistency', 'category_Fluency/Punctuation',
       'category_Fluency/Register', 'category_Fluency/Spelling',
       'category_Locale convention/Currency format',
       'category_Locale convention/Date format',
       'category_Locale convention/Time format', 'category_No-error',
       'category_Other', 'category_Source error', 'category_Style/Awkward',
       'category_Terminology/Inappropriate for context',
       'category_Terminology/Inconsistent',
       'category_Locale convention/Name format', 'category_Non-translation!',
       'category_Locale convention/Address format',
       'category_Locale convention/Telephone format', 'mqm_score', 's_mqm'])

Map:   0%|          | 0/5456 [00:00<?, ? examples/s]

Map:   0%|          | 0/9294 [00:00<?, ? examples/s]

# Training

In [ ]:
from transformers import TrainingArguments, Trainer

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr

def compute_metrics_for_regression(eval_pred):
    # print("eval_pred: ", eval_pred)
    logits, labels = eval_pred
    labels = labels.reshape(-1, 1)

    if np.all(labels == labels[0]) or np.all(logits == logits[0]):
      print(np.all(labels == labels[0]), np.all(logits == logits[0]))
      print(labels[0], logits[0])

    mse = mean_squared_error(labels, logits)
    rmse = mean_squared_error(labels, logits, squared=False)
    mae = mean_absolute_error(labels, logits)
    pearson_corr, _ = pearsonr(logits.flatten(), labels.flatten())

    return {"mse": mse, "rmse": rmse, "mae": mae, "pearson_corr": pearson_corr}

In [ ]:
class RegressionTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs[0][:, 0]
        loss = torch.nn.functional.mse_loss(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [ ]:
training_args = TrainingArguments(
    output_dir="../models/xlmr_multi_reg",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    metric_for_best_model="mse",
    load_best_model_at_end=True,
    weight_decay=0.01,
    remove_unused_columns=False
)

In [ ]:
from transformers import XLMRobertaForSequenceClassification

model = XLMRobertaForSequenceClassification.from_pretrained(BASE_MODEL,
                                                            num_labels = 1,
                                                            hidden_dropout_prob=0.2,
                                                            attention_probs_dropout_prob=0.2)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer = RegressionTrainer(
    model=model,
    args=training_args,
    train_dataset= ds["train"],
    eval_dataset= ds["validation"],
    compute_metrics=compute_metrics_for_regression,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Mse,Rmse,Mae,Pearson Corr
1,0.555000,0.461067,0.461067,0.679019,0.473917,0.733174
2,0.473700,0.417029,0.417029,0.645778,0.409676,0.768252
3,0.387300,0.429625,0.429625,0.655458,0.364517,0.785837
4,0.355900,0.382811,0.382811,0.618718,0.353130,0.791995


TrainOutput(global_step=5532, training_loss=0.4557418338819767, metrics={'train_runtime': 4413.4101, 'train_samples_per_second': 40.104, 'train_steps_per_second': 1.253, 'total_flos': 4.656918622481203e+16, 'train_loss': 0.4557418338819767, 'epoch': 4.0})

In [ ]:
trainer.eval_dataset = ds["test"]
trainer.evaluate()

{'eval_loss': 0.4542282223701477,
 'eval_mse': 0.4542282223701477,
 'eval_rmse': 0.6739645600318909,
 'eval_mae': 0.4773768186569214,
 'eval_pearson_corr': 0.7355576943625839,
 'eval_runtime': 108.8694,
 'eval_samples_per_second': 135.493,
 'eval_steps_per_second': 4.234,
 'epoch': 4.0}

In [ ]:
trainer.model.push_to_hub("xlmr_multi_reg")

pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/aningddd/xlmr_multi_reg/commit/6e9a2a9d57c6f2e4ca1869c6490aa0fc2dbfc437', commit_message='Upload XLMRobertaForSequenceClassification', commit_description='', oid='6e9a2a9d57c6f2e4ca1869c6490aa0fc2dbfc437', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
trainer.eval_dataset = single_lang["ende"]
trainer.evaluate()

{'eval_loss': 0.2839832007884979,
 'eval_mse': 0.28398317098617554,
 'eval_rmse': 0.5329006910324097,
 'eval_mae': 0.3444589376449585,
 'eval_pearson_corr': 0.6751161825898047,
 'eval_runtime': 40.3062,
 'eval_samples_per_second': 135.364,
 'eval_steps_per_second': 4.243,
 'epoch': 4.0}

In [ ]:
trainer.eval_dataset = single_lang["zhen"]
trainer.evaluate()

{'eval_loss': 0.5542186498641968,
 'eval_mse': 0.5542186498641968,
 'eval_rmse': 0.7444586157798767,
 'eval_mae': 0.5554553866386414,
 'eval_pearson_corr': 0.7264480039569028,
 'eval_runtime': 68.6493,
 'eval_samples_per_second': 135.384,
 'eval_steps_per_second': 4.239,
 'epoch': 4.0}

# Error Analysis

In [ ]:
import math

nb_batches = math.ceil(len(raw_ende_test_ds)/BATCH_SIZE)
y_preds = []

for i in range(nb_batches):
    input_source = raw_ende_test_ds[i * BATCH_SIZE: (i+1) * BATCH_SIZE]["source"]
    input_target = raw_ende_test_ds[i * BATCH_SIZE: (i+1) * BATCH_SIZE]["target"]
    input_labels = raw_ende_test_ds[i * BATCH_SIZE: (i+1) * BATCH_SIZE]["s_mqm"]
    encoded = tokenizer(input_source, input_target, truncation=True, padding="max_length", max_length=MAX_LEN, return_tensors="pt").to("cuda")
    y_preds += trainer.model(**encoded).logits.reshape(-1).tolist()

In [ ]:
pd.set_option('display.max_rows', 500)
ende_df = pd.DataFrame([raw_ende_test_ds["source"], raw_ende_test_ds["target"], raw_ende_test_ds["mqm_score"], (np.array(y_preds).reshape(-1, 1) * train_sd) + train_mean], ["Source", "Target", "Score", "Prediction"]).T
ende_df

,Source,Target,Score,Prediction
0,Iran reports lowest number of daily COVID-19 c...,Iran meldet<v> niedrigste</v> Zahl an tägliche...,1.0,[1.2244889460686588]
1,If the book still has issues to open after syn...,Falls das Buch nach der Synchronisation im E-R...,0.0,[-0.060719041105500526]
2,"""Russia should be in no doubt that further mil...","""Russland sollte keinen Zweifel daran haben, d...",0.1,[0.26329052502422834]
3,"I do apologise about this, as the account hold...","Ich entschuldige mich dafür, da der Kontoinhab...",1.0,[1.608242952944432]
4,"Beside 'Repair your #PRS_ORG# account', tap Re...","Neben ""Reparieren Sie Ihr # PRS _ ORG # -Konto...",1.0,[0.2647918679199377]
...,...,...,...,...
5451,Germany's regulator has suspended the approval...,Deutschlands Regulierungsbehörde hat das Geneh...,0.0,[-0.06567067512799563]
5452,Move the trailer!,Bewegen Sie den Anhänger!,0.0,[-0.2854812005591709]
5453,A rollercoaster first half ended with Munster ...,Eine Achterbahnfahrt der ersten Halbzeit endet...,5.0,[2.7015084394312403]
5454,"Best way to lose a new young worker, by shocki...",Der beste Weg einen neuen jungen Arbeiter zu v...,0.0,[-0.10315849230867014]


In [ ]:
# ende_df['Prediction'] = ende_df['Prediction'].apply(lambda x: float(x.strip('][')))
correlation, _ = pearsonr(ende_df['Score'], ende_df['Prediction'])
correlation

array([0.67511618])

In [ ]:
ende_df.to_csv("multi-reg-ende-pred.csv")

In [ ]:
zhen_batches = math.ceil(len(raw_zhen_test_ds)/BATCH_SIZE)
y_preds_zhen = []

for i in range(zhen_batches):
    input_source = raw_zhen_test_ds[i * BATCH_SIZE: (i+1) * BATCH_SIZE]["source"]
    input_target = raw_zhen_test_ds[i * BATCH_SIZE: (i+1) * BATCH_SIZE]["target"]
    input_labels = raw_zhen_test_ds[i * BATCH_SIZE: (i+1) * BATCH_SIZE]["s_mqm"]
    encoded = tokenizer(input_source, input_target, truncation=True, padding="max_length", max_length=MAX_LEN, return_tensors="pt").to("cuda")
    y_preds_zhen += trainer.model(**encoded).logits.reshape(-1).tolist()

In [ ]:
pd.set_option('display.max_rows', 500)
zhen_pred_df = pd.DataFrame([raw_zhen_test_ds["source"], raw_zhen_test_ds["target"], raw_zhen_test_ds["mqm_score"], (np.array(y_preds_zhen).reshape(-1, 1) * train_sd) + train_mean], ["Source", "Target", "Score", "Prediction"]).T
zhen_pred_df

,Source,Target,Score,Prediction
0,但是，时代变了。,But times have changed.,0.0,[-0.20626065126594462]
1,有意思的是，现在字节的TT 是Meta 主要的竞争对手之一，但是前两年Meta 从字节身上可...,"Interestingly, TT is one of the main competito...",5.0,[4.086536737027024]
2,我们召开党的十九届六中全会，总结党的百年奋斗重大成就和历史经验，通过百年党史上第三个历史决议...,We convened the Sixth Plenary Session of the 1...,5.0,[4.147207899548575]
3,奶奶的除夕夜说学逗唱二十四节气清冬见远山爷爷是个笨小孩我的奶奶住在古里古怪镇饺子和汤圆一块巧...,Grandma's New Year's Eve Sing and learn to sin...,1.0,[3.6418635896113516]
4,新华社利雅得12月12日电（记者王海洲胡冠）沙特阿拉伯首届当代艺术双年展— — 迪里耶当代艺...,Xinhua News Agency report of December 12 from ...,1.0,[3.190844766223148]
...,...,...,...,...
9289,“最好原则”的最大问题在于门槛太高，它往往超越了普通人的能力极限，让绝大多数人“望门兴叹” 。,The biggest problem with the “best principle” ...,0.1,[0.24377443506297558]
9290,去之前看了店里的拍摄作品，更多的是户外婚纱，拍的很漂亮，涉及到的拍摄风格也很多。,"Before I went there, I saw the shooting works ...",5.0,[4.204212428364579]
9291,马三立先生那段著名的《从明天开始》的相声，就是最传神地表现了这一“普遍人性”的经典。,Mr. Ma Sanli's famous comic “Starting Tomorrow...,5.0,[3.624866771679322]
9292,他同时认为，数字化转型需驱动向纵深发展和可持续发展，参与新冠肺炎疫情防控、经济复苏和发展、应...,He also believes that digital transformation n...,5.0,[3.6839362531091515]


In [ ]:
corr, _ = pearsonr(zhen_pred_df['Score'], zhen_pred_df['Prediction'])
corr

array([0.72644801])

In [ ]:
zhen_pred_df.to_csv("multi-reg-zhen-pred.csv")